In [ ]:
import math
import sys

import IPython
import IPython.display as ipd
import matplotlib.pylab as plt
import numpy as np
import pandas as pd

%reload_ext autoreload
%autoreload 2

%matplotlib inline
#%matplotlib notebook

from matplotlib import rcParams
rcParams["figure.max_open_warning"] = False

In [ ]:
def load_params(exp_name):
    """ load parameters module at the experiment of interest """
    import importlib.util
    spec = importlib.util.spec_from_file_location("params", f"../experiments/{exp_name}/params.py")
    params = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(params)
    return params

# Experimental distance-frequency matrix

# 1. Frequency slices

In [ ]:
exp_name = '2020_12_9_rotating'; 
fname = f'results/{exp_name}_real.pkl'

try:
    df_freq = pd.read_pickle(fname)
    print('read', fname)
except:
    print('Error: run wall_analysis.py to parse experiments.')
df_freq.tail()

In [ ]:
from frequency_analysis import add_spectrogram

df_freq = df_freq.assign(spectrogram=None)
df_freq = df_freq.apply(add_spectrogram, axis=1)

## a) analysis without snr-filtering

In [ ]:
from wall_analysis import filter_by_dicts
chosen_dicts = [{"appendix":"", "snr":0, "degree": 0}]
df_chosen = filter_by_dicts(df_freq, chosen_dicts)
df_chosen.tail()

### spectrograms

In [ ]:
mic_type = 'measurement'
cut_x = 500
cut_y = range(100, 400)

filters = ['source', 'degree', 'distance']
for chosen_tuple, df_source in df_chosen[df_chosen.mic_type==mic_type].groupby(filters):
    fig, axs = plt.subplots(1, 3, squeeze=False)
    fig.set_size_inches(15, 5)
    fig.suptitle(dict(zip(filters, chosen_tuple)))

    for j, motors in enumerate([0, "all43000"]):
        df_this = df_source[df_source.motors==motors]
        if len(df_this) != 1:
            continue
        row = df_this.iloc[0]
        
        spec = np.mean(row.spectrogram, axis=1) # n_times x n_mics x n_freqs
        axs[0, j].pcolorfast(row.seconds, row.frequencies, np.log(spec))
        axs[0, j].set_title(f'motors {motors}')

    axs[0, 1].plot([row.seconds[0], row.seconds[cut_x]], [row.frequencies[cut_y[0]], row.frequencies[cut_y[0]]], color='red', linewidth=1)
    axs[0, 1].plot([row.seconds[0], row.seconds[0]], [row.frequencies[cut_y[0]], row.frequencies[cut_y[-1]]], color='red', linewidth=1)
    axs[0, 1].plot([row.seconds[cut_x], row.seconds[cut_x]], [row.frequencies[cut_y[0]], row.frequencies[cut_y[-1]]], color='red', linewidth=1)
    axs[0, 1].plot([row.seconds[0], row.seconds[cut_x]], [row.frequencies[cut_y[-1]], row.frequencies[cut_y[-1]]], color='red', linewidth=1)
    
    axs[0, 2].pcolorfast(row.seconds[:cut_x], row.frequencies[cut_y], np.log(spec[cut_y, :cut_x]))
    #fig.savefig(f'/home/duembgen/Desktop/spec-{chosen_source}.png', bbox_inches='tight')

### frequency slices

In [ ]:
from bin_selection import select_frequencies
from constants import SPEED_OF_SOUND
from crazyflie_description_py.parameters import N_BUFFER
    
params_freq = load_params(exp_name)
    
CHOSEN_MICS = [0, 1, 2, 3]

for (distance,mic_type,motors), df_this in df_chosen.groupby(['distance', 'mic_type', 'motors']):
    if mic_type == 'audio_deck':
        mic_indices = CHOSEN_MICS
        bins_ = range(32)
    else:
        mic_indices = [0]
        bins_ = select_frequencies(N_BUFFER, 44100, min_freq=params_freq.MIN_FREQ, max_freq=params_freq.MAX_FREQ)

    fig, axs = plt.subplots(2, len(mic_indices), squeeze=False)
    fig.set_size_inches(15, 5)
    fig.suptitle(f'{distance}cm, {mic_type} mic, motors:{motors}')

    if len(df_this) != 1: 
        print(f"{len(df_this)} findings for {motors, chosen_source}")
        continue

    row = df_this.iloc[0]
    stft = row.stft # times x n_mics x frequencies
    frequencies = row.frequencies
    
    mean_f = frequencies[bins_[1]] - frequencies[bins_[0]] 
    expected_period = SPEED_OF_SOUND / (2 * distance * 1e-2) # m/s / m = Hz

    for mic_idx in mic_indices:
        spec = np.sum(np.abs(stft[:, mic_idx, bins_]), axis=0) # times x frequencies
        axs[0, mic_idx].semilogy(frequencies[bins_], spec, label=f'mic{mic_idx}', color=f'C{mic_idx}')
        axs[0, mic_idx].set_title(f'm{mic_idx}')

        spec_fft = np.fft.rfft(spec)[1:]
        freqs = np.fft.rfftfreq(len(spec), mean_f)[1:] # "seconds"
        axs[1, mic_idx].plot(1/freqs, np.abs(spec_fft), color=f'C{mic_idx}')
        axs[1, mic_idx].axvline(expected_period, color=f'C{mic_idx}', ls=":") # cm

    #axs[0, mic_idx].legend(loc='upper right')
    #axs[0, mic_idx].set_xlim(0,50)
    #axs[1, mic_idx].set_xlim(1e-1,50)
    #axs[0, j].set_ylim(min(spec), max(spec))

## b) analysis with snr-filtering

In [ ]:
from wall_analysis import filter_by_dicts
from crazyflie_description_py.parameters import N_BUFFER, FS

snr = 1
appendix = ""
chosen_dicts = [{"appendix":appendix, "snr":snr, "degree": 0}]
df_chosen = filter_by_dicts(df_freq, chosen_dicts)
df_chosen.tail()

all_frequencies = np.fft.rfftfreq(N_BUFFER, 1/FS)

### spectrograms

In [ ]:
mic_type = 'audio_deck'
filters = ['source', 'degree', 'distance']

MIN_FREQ = 1000
MAX_FREQ = 5000

for chosen_tuple, df_source in df_chosen[df_chosen.mic_type==mic_type].groupby(filters):
    fig, axs = plt.subplots(1, 2, squeeze=False, sharex=True, sharey=True)
    fig.set_size_inches(15, 5)
    fig.suptitle(dict(zip(filters, chosen_tuple)))
    axs[0, 0].set_ylabel('frequency [Hz]')
    for j, motors in enumerate(params_freq.MOTORS_LIST):
        df_this = df_source[df_source.motors==motors]
        if len(df_this) != 1:
            continue
        row = df_this.iloc[0]
        
        for i in range(3):
            #axs[0, j].scatter(row.seconds, row.frequencies_matrix[:, i], label=i)
            axs[0, j].scatter(range(len(row.seconds)), row.frequencies_matrix[:, i], label=f"{i+1}-strongest")
            
        axs[0, j].set_xlabel('time idx [-]')
        axs[0, j].set_title(f'motors {motors}')
        axs[0, j].legend(loc="upper right")
        
    fig, axs = plt.subplots(1, 2, squeeze=False, sharex=True, sharey=True)
    fig.set_size_inches(15, 5)
    fig.suptitle(dict(zip(filters, chosen_tuple)))
    axs[0, 0].set_ylabel('frequency [Hz]')
    for j, motors in enumerate(params_freq.MOTORS_LIST):
        df_this = df_source[df_source.motors==motors]
        if len(df_this) != 1:
            continue
        row = df_this.iloc[0]
        spec = np.mean(row.spectrogram, axis=1)
        all_frequencies = np.unique(row.frequencies_matrix)
        axs[0, j].pcolorfast(row.seconds, all_frequencies, np.log10(spec[:-1, :-1]))
        axs[0, j].set_xlabel('time [s]')
        
        axs[0, j].set_title(f'motors {motors}')
        axs[0, j].set_ylim(MIN_FREQ, MAX_FREQ)

## frequency slices

In [ ]:
from simulation import get_setup

distance = 10

fig = plt.figure()
fig.set_size_inches(5, 5)
for yaw_deg, marker in zip([0, 30], ['o', 'x']):
    source, mic_positions = get_setup(distance_cm=10, yaw_deg=yaw_deg)
    source_image = [source[0], -source[1]]
    for i, mic in enumerate(mic_positions):
        plt.scatter(*mic, label=f'mic{i}, {yaw_deg}deg', marker=marker, color=f'C{i}')
plt.plot([4.8, 5.4], [0, 0], color='k', label='wall')
plt.title(f'setup for distance={distance}cm')
plt.legend(loc='upper right')
plt.xlabel('x [m]')
plt.ylabel('y [m]')
plt.axis('equal')

fig.savefig(f'plots/setup_{distance}.pdf', bbox_inches='tight')

In [ ]:
from scipy.interpolate import interp1d

CHOSEN_MICS = [0, 1, 2, 3] #[0, 1, 2, 3] #[3]

calib_df = pd.read_pickle('results/calibration_results.pkl')
chosen_dict = {
   'source': 'sweep',
   #'method': 'median-reject',
   'method': 'median',
   'type': 'psd_df',
}
calib_df = filter_by_dicts(calib_df, [chosen_dict])
assert len(calib_df) == 1
row = calib_df.iloc[0]

fig, axs = plt.subplots(1, len(CHOSEN_MICS), squeeze=False, sharey=True, sharex=True)
fig.set_size_inches(10, 5)
#fig.suptitle(f'{distance}cm, {mic_type} mic, motors:{motors}')
for i, mic_idx in enumerate(CHOSEN_MICS):
    axs[0, i].semilogy(row.frequencies, row.psd[mic_idx], color=f'C{mic_idx}')
    axs[0, i].set_xlabel('frequency [Hz]')
    axs[0, i].set_title(f'mic{mic_idx}')
axs[0, 0].set_ylabel('PSD')
axs[0, i].set_ylim(1e-2, 3e2)

calib_psd = row.psd
calib_f = row.frequencies

uniform_f = np.linspace(1000, 6000, 50)
calib = interp1d(x=calib_f, y=calib_psd, kind='linear', fill_value='extrapolate')
uniform_psd = calib(uniform_f)
for i, mic_idx in enumerate(CHOSEN_MICS):
    axs[0, i].scatter(uniform_f, uniform_psd[mic_idx], color=f'C{mic_idx}')
    axs[0, i].grid('both')
    
fig.savefig('plots/calibration.pdf', bbox_inches='tight')

In [ ]:
from constants import SPEED_OF_SOUND
from frequency_analysis import extract_psd_dict, extract_psd
from simulation import get_freq_slice_pyroom, get_freq_slice_theory
    
MIN_FREQ = 1000
MAX_FREQ = 4000

filter_dict = {
    'mic_type': 'audio_deck',
    'degree': 0
}
df_audio_deck = filter_by_dicts(df_chosen, [filter_dict])

distances = df_chosen.distance.unique()
features = {
    'variance': np.zeros((4, len(distances))),
    'range': np.zeros((4, len(distances)))
}

try:
    signal = pd.read_pickle('results/multi.pk')
except FileNotFoundError:
    print('Run WallStudy notebook to save signal results/multi.pk')

d_idx = 0
for (distance, motors), df_this in df_audio_deck.groupby(['distance', 'motors']):
    if motors != 0:
        continue
        
    if len(df_this) != 1: 
        print(f"{len(df_this)} findings for {distance}, {mic_type}, {motors}")
        continue

    row = df_this.iloc[0]
    
    psd_dict = extract_psd_dict(row.stft, row.frequencies_matrix, 
                                min_t=60, max_t=190, n_freq=n_freq)
    psd, frequencies, psd_std = extract_psd(psd_dict)
    psd_calib = calib(frequencies)
    #psd_mean = np.mean(psd_calib, axis=0)
    
    mask = (frequencies < MAX_FREQ) & (frequencies > MIN_FREQ)
    psd = psd[:, mask]
    psd_calib = psd_calib[:, mask]
    #psd_mean = psd_mean[mask]
    frequencies = frequencies[mask]
    
    source, mic_positions = get_setup(distance_cm=distance)
    source_image = [source[0], -source[1]]
    
    fig, axs = plt.subplots(3, len(CHOSEN_MICS), squeeze=False, sharey=True, sharex=True)
    axs[0, 0].set_yscale('log')
    axs[0, 0].set_ylim(1e-2, 3e2)
    fig.set_size_inches(10, 10)
    
    slices_theory = get_freq_slice_theory(frequencies, distance_cm=distance)
    slices_pyroom = get_freq_slice_pyroom(frequencies, distance_cm=distance, signal=signal)
    
    for j, mic_idx in enumerate(CHOSEN_MICS):
        axs[0, j].set_title(f'm{mic_idx}')
        axs[0, j].plot(frequencies, psd[mic_idx], label='|Y(f)|', color=f'C{mic_idx}')
        axs[0, j].plot(frequencies, psd_calib[mic_idx], ls=':', label='|X(f)|', color=f'C{mic_idx}')
        #axs[0, j].plot(frequencies, psd_mean, color='k')
        
        psd_ratio = psd[mic_idx]/psd_calib[mic_idx]
        axs[1, j].plot(frequencies, psd_ratio, label=f'|H(f)|', color=f'C{mic_idx}')
        #axs[1, j].plot(frequencies, psd[mic_idx]/psd_mean, label=f'global calib', color='k')
        
        features['variance'][mic_idx, d_idx] = np.var(psd_ratio)
        features['range'][mic_idx, d_idx] = np.max(psd_ratio) - np.min(psd_ratio)
        axs[0, j].legend(loc='lower right')
        axs[1, j].legend(loc='lower right')

        axs[2, j].plot(frequencies, np.abs(slices_theory[mic_idx]), label='theory', color="black")
        axs[2, j].plot(frequencies, np.abs(slices_pyroom[mic_idx]), label='pyroom', color="gray")
        axs[2, j].legend(loc='lower right')
        
        axs[0, j].grid('both')
        axs[1, j].grid('both')
        axs[2, j].grid('both')
        
    fig.suptitle(f'{distance}cm, {mic_type} mic, motors:{motors}', y=0.93)
    fname = f'plots/f_slice_{distance}.pdf'
    fig.savefig(fname, bbox_inches='tight')
    print('saved as', fname)

    d_idx += 1
    continue
    fig, axs = plt.subplots(1, len(CHOSEN_MICS), squeeze=False, sharey=True, sharex=True)
    #axs[0, 0].set_yscale('log')
    #axs[0, 0].set_ylim(1e-1, 20)
    fig.set_size_inches(15, 5)
    fig.suptitle(f'{distance}cm, {mic_type} mic, motors:{motors}')
    for j, mic_idx in enumerate(CHOSEN_MICS):
        mic = mic_positions[mic_idx]
        delta = np.linalg.norm(mic - source_image) - np.linalg.norm(mic - source)
        expected_period = SPEED_OF_SOUND / delta # m/s / m = Hz
        psd_ratio = psd[mic_idx]/psd_calib[mic_idx]
        psd_fft = np.fft.rfft(psd_ratio)[1:]
        mean_f = np.mean(frequencies[1:]-frequencies[:-1])
        freqs = np.fft.rfftfreq(len(psd_ratio), mean_f)[1:] # "seconds"
        axs[0, j].semilogx(1/freqs, np.abs(psd_fft), color=f'C{mic_idx}')
        axs[0, j].axvline(expected_period, color=f'C{mic_idx}', ls=":") # cm

In [ ]:
fig, axs = plt.subplots(1, len(features), squeeze=False)
for j, (key, vals) in enumerate(features.items()):
    for i in range(vals.shape[0]):
        axs[0, j].semilogy(distances, vals[i], label=f'mic{i}', color=f'C{i}', marker='x')
    #axs[0, j].set_title(key)
    axs[0, j].set_xlabel('distance [cm]')
    axs[0, j].set_ylabel(f'feature {key}')

# Distance slices

In [ ]:
#exp_name = '2020_12_7_moving'; 
exp_name = '2020_12_18_stepper'; 
fname = f'results/{exp_name}_real.pkl'

params_dist = load_params(exp_name)

try:
    df_dist = pd.read_pickle(fname)
    print('read', fname)
except:
    print('Error: run wall_analysis.py to parse experiments.')

In [ ]:
from wall_analysis import add_distance_estimates
from frequency_analysis import add_spectrogram

df_dist = df_dist.assign(d_estimate=None)
df_dist = df_dist.apply(add_distance_estimates, axis=1)

df_dist = df_dist.assign(spectrogram=None)
df_dist = df_dist.apply(add_spectrogram, axis=1)
df_dist

## sanity checks

In [ ]:
import itertools
from wall_analysis import filter_by_dicts
labels = ['motors', 'source']
vals_dict = {l:df_dist[l].unique() for l in labels}
vals_dict['source'] = sorted(vals_dict['source'])
vals_dict['motors'] = sorted([m for m in vals_dict['motors'] if type(m) is int]) + sorted([m for m in vals_dict['motors'] if not type(m) is int])
print(vals_dict)

fig, axs = plt.subplots(1, len(df_dist.mic_type.unique()))
fig.set_size_inches(10, 5)
axs = dict(zip(df_dist.mic_type.unique(), axs))
fig.suptitle('timestamps for different datasets')
for mic_type, df_mic in df_dist.groupby('mic_type'):
    for i, val_tuple in enumerate(itertools.product(*vals_dict.values())):
        filter_dict = dict(zip(labels, val_tuple))
        df_here = filter_by_dicts(df_mic, [filter_dict])
        
        label = f"{list(filter_dict.values())}"
        if len(df_here) != 1:
            axs[mic_type].plot([], [], color=f'C{i%10}', label=label + ': missing')
            continue
        row = df_here.iloc[0]
        axs[mic_type].plot(row.seconds, row.seconds+i*10, color=f'C{i%10}', label=label)
    axs[mic_type].set_title(mic_type)
    axs[mic_type].legend(bbox_to_anchor=[1.0, -0.1], loc='upper right')
    
fig, axs = plt.subplots(1, len(df_dist.mic_type.unique()))
fig.set_size_inches(10, 5)
axs = dict(zip(df_dist.mic_type.unique(), axs))
fig.suptitle('distance estimates for different datasets')
for mic_type, df_mic in df_dist.groupby('mic_type'):
    for i, val_tuple in enumerate(itertools.product(*vals_dict.values())):
        filter_dict = dict(zip(labels, val_tuple))
        df_here = filter_by_dicts(df_mic, [filter_dict])
        
        label = f"{list(filter_dict.values())}"
        if not len(df_here):
            axs[mic_type].plot([], [], color=f'C{i%10}', label=label + ': missing')
            continue
        elif len(df_here) > 2:
            raise ValueError(len(df_here))
        row = df_here.iloc[0]
        axs[mic_type].plot(row.seconds, row.d_estimate+i/100, color=f'C{i%10}', label=label)
    axs[mic_type].set_title(mic_type)
    axs[mic_type].legend(bbox_to_anchor=[1.0, -0.1], loc='upper right')

In [ ]:
from wall_analysis import filter_by_dicts
print(exp_name)
if exp_name == "2020_12_7_moving":
    motors = "all43000"
    chosen_dicts = [
        {"source": "mono3125", "motors": 0, "appendix":""},
        {"source": "mono3125", "motors": motors, "appendix":"_new"},
        {"source": "mono4156", "motors": 0, "appendix":""},
        {"source": "mono4156", "motors": motors, "appendix":"_new"},
        {"source": "mono8000", "motors": 0, "appendix":"_new"},
        {"source": "mono8000", "motors": motors, "appendix":""},
        {"source": "None", "motors": motors, "appendix":"_new"},
    ]
    df_chosen = filter_by_dicts(df_dist, chosen_dicts)
else: 
    chosen_dicts = [
        #{"motors": 0, "source": s} for s in df_dist.source.unique() if s is not None
        {}
    ]
    df_chosen = filter_by_dicts(df_dist, chosen_dicts)
print(df_chosen.source.unique())

## spectrograms

In [ ]:
def plot_square(xs, ys, x_indices, y_indices, ax):
    ax.plot([xs[x_indices[0]], xs[x_indices[-1]]], [ys[y_indices[-1]], ys[y_indices[-1]]], color='red', linewidth=1)
    ax.plot([xs[x_indices[0]], xs[x_indices[-1]]], [ys[y_indices[0]], ys[y_indices[0]]], color='red', linewidth=1)
    ax.plot([xs[x_indices[0]], xs[x_indices[0]]], [ys[y_indices[0]], ys[y_indices[-1]]], color='red', linewidth=1)
    ax.plot([xs[x_indices[-1]], xs[x_indices[-1]]], [ys[y_indices[0]], ys[y_indices[-1]]], color='red', linewidth=1)

mic_type = 'measurement'
cut_x = range(0, 300)
cut_y = range(10, 200)

filters = ['source', 'degree']
for chosen_tuple, df_source in df_chosen[df_chosen.mic_type==mic_type].groupby(filters):
    filter_dict = dict(zip(filters, chosen_tuple))
    fig, axs = plt.subplots(1, 3, squeeze=False)
    fig.set_size_inches(15, 5)
    title = ''
    for key, val in filter_dict.items():
        title += f'{key}: {val}, '
        break
    title = title[:-2]
    fig.suptitle(title)
    
    j_square = None
    for j, motors in enumerate(params_dist.MOTORS_LIST):
        df_this = df_source[df_source.motors==motors]
        if len(df_this) != 1:
            continue
        row = df_this.iloc[0]
        spec = np.mean(row.spectrogram, axis=1)

        axs[0, j].pcolorfast(row.seconds, row.frequencies, np.log10(spec))
        axs[0, j].set_title(f'motors {motors}')
        j_square = j
        
    if j_square is not None:
        plot_square(row.seconds, row.frequencies, cut_x, cut_y, axs[0, j_square]) 
        spec_square  = spec[:, cut_x]
        spec_square = spec_square[cut_y, :]
        axs[0, 2].pcolormesh(row.seconds[cut_x], row.frequencies[cut_y], np.log10(spec_square))
    axs[0, 0].set_xlabel('time idx')
    axs[0, 0].set_ylabel('frequency [Hz]')
    axs[0, 1].set_xlabel('time idx')
    axs[0, 1].set_ylabel('frequency [Hz]')
    axs[0, 2].set_xlabel('time idx')
    axs[0, 2].set_ylabel('frequency [Hz]')
    
    #fname = f'plots/spec_{filter_dict["source"]}.pdf' 
    #fig.savefig(fname, bbox_inches='tight')
    #print('saved as', fname)

## frequency selection for propeller noise

In [ ]:
df_source = df_chosen.loc[df_chosen.source=="None"]
df_mic = df_source.loc[df_source.mic_type == "audio_deck"]
assert len(df_mic) == 1
row = df_mic.iloc[0]

f_matrix = row.frequencies_matrix[:, 1:]
print(f_matrix.shape)
fig, ax = plt.subplots()
fig.set_size_inches(10, 5)
for i, col in enumerate(f_matrix.T):
    if i < 3:
        plt.scatter(row.seconds, col, color=f'C{i}', s=2)
    else:
        plt.scatter(row.seconds, col, color='C1', s=1, alpha=0.5)
plt.ylim(100, 2000)

In [ ]:
#levels = list(range(2))
levels = list(range(5))
num_levels = len(levels)
times = range(f_matrix.shape[0]) # range(100, 300)
#times = range(10, 800)
f_matrix_reduced = f_matrix[times, :]
f_matrix_reduced = f_matrix_reduced[:, levels]
fig, axs = plt.subplots(num_levels, sharey=False, sharex=True, squeeze=False)
fig.set_size_inches(10, 10)
for j, level in enumerate(levels):
    axs[j, 0].plot(times, f_matrix_reduced[:, j], color=f"C{j}", marker='o')
    freq = np.median(f_matrix_reduced[:, j])
    axs[j, 0].set_title(f'level {level}: {freq}Hz')

In [ ]:
all_freqs = np.sort(f_matrix[times,:10].flatten())
count, bins = np.histogram(all_freqs, bins=np.unique(all_freqs))
plt.plot(bins[:-1], count, marker='x')
plt.title('histogram strongest 10 frequency bins')
print(bins[np.argsort(count)[::-1]])

In [ ]:
all_freqs = np.sort(f_matrix[times,0].flatten())
count, bins = np.histogram(all_freqs, bins=np.unique(all_freqs))
plt.plot(bins[:-1], count)
plt.title('histogram strongest frequency bin')
print(bins[np.argsort(count)[::-1]])

## distance-slices

In [ ]:
#mic_indices = range(4)
from constants import SPEED_OF_SOUND

CHOSEN_MICS = [0, 1, 2, 3] #[3]
CUT_LAST = 3
CUT_FIRST = 3

#CHOSEN_PROP_FREQ = 781 #
CHOSEN_PROP_FREQ = 671 #
#CHOSEN_PROP_FREQ = 2187 #1

#CHOSEN_PROP_FREQ = 1453 

for chosen_source, df_source in df_chosen.groupby('source'):
    
    #if chosen_source in ['sweep_fast', 'sweep_slow']:
    #    continue
    
    for mic_type, df_this in df_source.groupby('mic_type'):
            
        if mic_type == 'audio_deck':
            mic_indices = CHOSEN_MICS
        else:
            mic_indices = [0]
            
        fig, axs = plt.subplots(1, 2, squeeze=False)
        fig.set_size_inches(15, 5)
        fig.suptitle(chosen_source)

        for j, motors in enumerate(params_dist.MOTORS_LIST):
            df_motors = df_this.loc[df_this.motors==motors]
            
            if len(df_motors) != 1: 
                print(f"{len(df_motors)} findings for {chosen_source, motors, mic_type}")
                continue
            row = df_motors.iloc[0]
            
                
            if (chosen_source == "None"):
                freq = CHOSEN_PROP_FREQ
            elif "mono" in chosen_source:
                freq = int(chosen_source.strip('mono'))
            else:
                freq = 1000
                
            if (row.snr == 2) and (chosen_source == "None") and (mic_type == "audio_deck"):
                bin_ = np.argmin(np.abs(row.frequencies_matrix - freq), axis=1)
            elif (row.snr == 2) and (chosen_source != "None") and (mic_type == "audio_deck"):
                bin_ = 0
            elif row.snr == 1:
                print(f"snr==1 for {chosen_source, motors, mic_type}")
                bin_ = np.argmin(np.abs(row.frequencies_matrix - freq), axis=1)
            else:
                print(f"snr==0 for {chosen_source, motors, mic_type}")
                bin_ = np.argmin(abs(row.frequencies - freq))
                error = abs(row.frequencies[bin_] - freq)
                if error > 10:
                    print(f'Warning: big diff between {freq} and {row.frequencies[bin_]}')

            #seconds = row.seconds[CUT_FIRST:-CUT_LAST]
            #distances = seconds * 50 / 165.0
            distances = row.d_estimate[CUT_FIRST:-CUT_LAST] * 100
            
            for mic_idx in mic_indices:
                spec = np.abs(row.stft[range(row.stft.shape[0]), mic_idx, bin_])[CUT_FIRST:-CUT_LAST] # times x frequencies
                axs[0, j].plot(distances, spec, label=f'mic{mic_idx}', color=f'C{mic_idx}')
                axs[0, j].set_title(f'motors {motors}')
                
            axs[0, j].set_ylim(min(spec), max(spec))
            axs[0, j].legend(loc='upper right')
            axs[0, j].set_xlim(10,60)
            axs[0, j].set_ylim(0,min(1, np.max(spec)))
            continue
            
            mean_d = np.median(distances[5:]-distances[4:-1])
            #print(mean_d)
            expected_period = SPEED_OF_SOUND / freq * 100 / 2 #
            for mic_idx in mic_indices:
                spec_fft = np.fft.rfft(spec)[1:]
                freqs = np.fft.rfftfreq(len(spec), mean_d)[1:]
                period = 1/freqs
                axs[1, j].semilogx(period, np.abs(spec_fft), color=f'C{mic_idx}')
                axs[1, j].axvline(expected_period, color=f'C{mic_idx}', ls=":") # cm
            axs[1, j].set_xlim(1e-1,50)

## study chosen datasets more carefully

In [ ]:
frequency = 3125 
#frequency = 2375
filter_dict = {
    'source': f'mono{frequency}',
    'motors': 0,
    #'motors': 'all45000',
    'mic_type': 'audio_deck',
    #'mic_type': 'measurement'
}
df_chosen = filter_by_dicts(df_dist, [filter_dict])
assert len(df_chosen) == 1, len(df_chosen)
row = df_chosen.iloc[0]
n_mics = row.stft.shape[1]

In [ ]:
from simulation import get_dist_slice_pyroom, get_dist_slice_theory
# 2. study the bin of interest
# chosen bin based on buzzer
if row.frequencies_matrix is not None:
    buzzer_times = row.frequencies_matrix[:, 0] == frequency
    valid_times = row.seconds > 0
    mask = buzzer_times & valid_times
    bin_ = 0
else:
    valid_times = row.seconds > 0
    mask = valid_times
    bin_ = np.argmin(np.abs(row.frequencies - frequency))

spec = np.abs(row.stft) # times x mics x freqs
n_mics = spec.shape[1]

distances_cm = np.linspace(10, 60)
slices_theory = get_dist_slice_theory(frequency, distances_cm=distances_cm)
slices_pyroom = get_dist_slice_pyroom(frequency, distances_cm=distances_cm)

fig1, axs1 = plt.subplots(1, n_mics, squeeze=False, sharey=True)
fig2, axs2 = plt.subplots(1, n_mics, squeeze=False, sharey=True)
fig1.set_size_inches(10, 5)
fig2.set_size_inches(10, 5)
for mic_idx in range(n_mics):
    
    axs1[0, mic_idx].set_title(f'mic{mic_idx}')
    axs1[0, mic_idx].semilogy(row.d_estimate[mask], spec[mask, mic_idx, bin_], 
                      color=f'C{mic_idx}', label='|Y(d)|')
    axs1[0, mic_idx].grid(which='both')
    axs1[0, mic_idx].legend(loc='upper left')
    axs1[0, mic_idx].set_xlabel('distance [m]')
    
    
    axs2[0, mic_idx].semilogy(distances_cm / 100, slices_theory[:, mic_idx], color='black', label='theory')
    axs2[0, mic_idx].semilogy(distances_cm / 100, slices_pyroom[:, mic_idx], color='gray', label='pyroom')
    axs2[0, mic_idx].grid(which='both')
    axs2[0, mic_idx].set_xlabel('distance [m]')
    axs2[0, mic_idx].legend(loc='upper left')
    axs2[0, mic_idx].set_ylim(2, 20)
    
    # custom stuff
    if frequency == 2375:
        if row.mic_type == 'measurement':
            axs1[0, mic_idx].set_ylim(1e-3, 1e-1)
            axs1[0, mic_idx].set_xlim(0.2, 0.4)
            axs2[0, mic_idx].set_xlim(0.2, 0.4)
        elif row.mic_type == 'audio_deck':
            axs1[0, mic_idx].set_ylim(1e-3, 5)
            axs1[0, mic_idx].set_xlim(0.3, 0.5)
            axs2[0, mic_idx].set_xlim(0.3, 0.5)
    elif frequency == 3125:
        if row.mic_type == 'measurement':
            axs1[0, mic_idx].set_ylim(1e-3, 1e-1)
            axs1[0, mic_idx].set_xlim(0.1, 0.3)
            axs2[0, mic_idx].set_xlim(0.1, 0.3)
        elif row.mic_type == 'audio_deck':
            axs1[0, mic_idx].set_ylim(1e-3, 5)
            axs1[0, mic_idx].set_xlim(0.3, 0.5)
            axs2[0, mic_idx].set_xlim(0.3, 0.5)
    
fig1.suptitle(f'{frequency} Hz, {row.mic_type} mic, motors:{row.motors}')
fname = f'plots/d_slice_{frequency}_{row.mic_type}_{row.motors}.pdf'
fig1.savefig(fname, bbox_inches='tight')

fname = f'plots/d_slice_{frequency}_{row.mic_type}_{row.motors}_theory.pdf'
fig2.savefig(fname, bbox_inches='tight')
print('saved as', fname)

In [ ]:
# 0. clean stft
from wall_analysis import clean_stft 
stft = row.stft
stft[np.isnan(stft)] = 0
print(np.min(np.abs(stft)), np.max(np.abs(stft)))
print(np.min(stft), np.max(stft))
print(np.nanmin(stft), np.nanmax(stft))

In [ ]:
# 1. study the spectrum, frequency selection
from crazyflie_description_py.parameters import N_BUFFER, FS
if row.mic_type == 'audio_deck':
    all_frequencies = np.fft.rfftfreq(N_BUFFER, 1/FS)
else:
    all_frequencies = np.fft.rfftfreq(N_BUFFER, 1/44100)
all_frequencies = np.unique(row.frequencies_matrix)
spec = np.nanmean(row.spectrogram, axis=1)

fig, ax = plt.subplots()
plot_spec = np.full(spec.shape, np.nan)
plot_spec[spec>0] = np.log10(spec[spec>0])
print(np.nanmin(plot_spec), np.nanmax(plot_spec))
ax.pcolorfast(range(len(row.seconds)), all_frequencies, plot_spec[:-1, :-1])
ax.axhline(frequency, color='black', ls=':')
ax.set_ylim(100, 5000)

fig, axs = plt.subplots(1, n_mics, squeeze=False)
fig.set_size_inches(10, 5)
for mic_idx in range(n_mics):
    spec_avg = np.nanmean(row.spectrogram[:, mic_idx, 100:500], axis=-1)
    axs[0, mic_idx].plot(all_frequencies, spec_avg, color=f'C{mic_idx}')
    axs[0, mic_idx].set_xlim(1000, 4000)
    axs[0, mic_idx].axvline(frequency, color='black', ls=':')

# Old experiments


In [ ]:
from evaluate_data import read_df, integrate_yaw
from dynamic_analysis import add_pose_to_df
from frequency_analysis import get_psd

#exp_name = '2020_11_23_wall2'; DISTANCE_LIST = np.arange(100, step=10)
exp_name = '2020_11_26_wall'; DISTANCE_LIST = np.arange(50)

#fname = f'results/{exp_name}_simulated.pkl'
fname = f'results/{exp_name}_real.pkl'

SOURCE_LIST = ['mono4125', 'mono3500', None, 'sweep', 'sweep_low', 'sweep_high'] # 
DEGREE_LIST = [0, 27, 54, 81, 360]

try:
    df_total = pd.read_pickle(fname)
    frequencies = df_total.iloc[0].frequencies
    print('read', fname)
except:
    print('could not read', fname)
    df_total = pd.DataFrame(columns=['stft', 'degree', 'yaw', 'distance', 'source', 'psd', 'spec', 
                                     'frequencies'])

    params = dict(
      props = False,
      snr = False,
      motors = False,
      exp_name = exp_name
    )
    
    for degree in DEGREE_LIST:
        for distance in DISTANCE_LIST:
            for source in SOURCE_LIST:
                try:
                    params['degree'] = degree
                    params['distance'] = distance
                    params['source'] = source
                    params['appendix'] = ""
                    if (exp_name == '2020_11_23_wall2') and (distance in [10, 30, 50]):
                        params['appendix'] = "_new"
                    df, df_pos = read_df(**params)
                except Exception as e:
                    continue 

                # detect index decrease (happens when two csv files are concatenated)
                sign = np.sign(df['index'].values[1:] - df['index'].values[:-1])
                if np.any(sign < 0):
                    index = np.where(sign<0)[0][-1]
                    print('Warning: found multiple start indices, start at', index)
                    df = df.iloc[index:]
                    index_start = df.iloc[0]['index']
                    df_pos = df_pos.loc[df_pos.index >= index_start]

                stft = np.array([*df.signals_f.values]) # n_times x n_mics x n_freqs
                frequencies_matrix = np.array([*df.loc[:,'frequencies']])
                frequencies = frequencies_matrix[0, :]
                assert not np.any(np.any(frequencies_matrix - frequencies[None, :], axis=0))

                if degree == 360:
                    add_pose_to_df(df, df_pos, max_allowed_lag_ms=50)
                    yaw = integrate_yaw(df.timestamp.values, df.yaw_rate_deg.values)
                else:
                    yaw = np.full(len(df), -degree)

                spec = np.sum(np.abs(stft), axis=1)
                psd = get_psd(stft, frequencies, fname='real')

                df_total.loc[len(df_total), :] = dict(
                    degree=degree,
                    yaw=yaw,
                    distance=distance,
                    source=str(source),
                    stft=stft,
                    frequencies=frequencies,
                    spec=spec,
                    psd=psd
                )

    fname = f'results/{exp_name}_real.pkl'
    pd.to_pickle(df_total, fname)
    print('saved as', fname)

In [ ]:
#for deg in [0, 27, 54, 81]:
    #df_total.loc[df_total.degree==deg, 'source'] = 'sweep'

In [ ]:
n_mics = df_total.iloc[0].stft.shape[1]
df_matrix_mics = np.empty((n_mics, len(frequencies), len(DISTANCE_LIST)))
normalize = False

if normalize:
    psd_ref = df_total[df_total.distance==DISTANCE_LIST[-1]].iloc[0].psd

for j, distance_cm in enumerate(DISTANCE_LIST):
    row = df_total[df_total.distance==distance_cm].iloc[0]
    for i in range(n_mics):
        if normalize:
            df_matrix_mics[i, :, j] = row.psd[i] / psd_ref[i]
        else:
            df_matrix_mics[i, :, j] = row.psd[i]

In [ ]:
fig, axs = plt.subplots(1, n_mics)
fig.set_size_inches(10, 5)
for i in range(n_mics):
    axs[i].pcolormesh(DISTANCE_LIST, frequencies, np.log(df_matrix_mics[i]))

In [ ]:
from constants import SPEED_OF_SOUND
mic_idx = 0
for f_idx, slice_f in enumerate(np.arange(32)):
    
    fig, axs = plt.subplots(2, n_mics)
    fig.set_size_inches(15, 5)
    
    f = frequencies[slice_f]
    expected_period = SPEED_OF_SOUND / f * 1e2 / 2
    
    for mic_idx in range(n_mics):
        distance_response = df_matrix_mics[mic_idx, slice_f, :]
        axs[0, mic_idx].semilogy(DISTANCE_LIST, distance_response, label=f'{f:.0f}Hz')
        axs[0, mic_idx].set_xlabel('distance [cm]')
        axs[0, mic_idx].set_title(f'mic{mic_idx}')
        axs[0, mic_idx].legend(loc='lower left')

        distance_fft = np.fft.rfft(distance_response)[1:]
        distance_freq = 1 / np.fft.rfftfreq(len(DISTANCE_LIST), 1)[1:] # cm 
        axs[1, mic_idx].loglog(distance_freq, np.abs(distance_fft))
        axs[1, mic_idx].axvline(expected_period, ls=":")
        axs[1, mic_idx].set_xlabel('period [cm]')
    
    axs[0, 0].set_ylabel('PSD')

## Fixed angle analysis

In [ ]:
from wall_analysis import filter_by_dicts
try:
    filter_dict = dict(
        distance=40,
        degree=0,
        source='sweep'
    )
    df = filter_by_dicts(df_total, [filter_dict])
    row = df.iloc[0]
    
    spec = np.sum(np.abs(row.stft), axis=1)
    
    plt.figure()
    times = np.arange(spec.shape[0])
    plt.pcolormesh(times, frequencies, np.log10(spec.T))

    psd = get_psd(row.stft, frequencies, ax=plt.gca(), fname='real')

    plt.figure()
    for i_mic in range(psd.shape[0]):
        plt.semilogy(frequencies, np.abs(psd[i_mic, :]), label=f"mic{i_mic}")
    plt.xlabel('frequency [Hz]')
    plt.ylabel('PSD')
    plt.title(filter_dict)
except ValueError:
    raise
except Exception as e:
    print(f'did not find {filter_dict} in')
    print(df_total.distance.unique())
    print(df_total.degree.unique())
    print(df_total.source.unique())
    print(e)

In [ ]:
degrees = [d for d in df_total.degree.unique() if d != 360][:2]
distances = df_total.distance.unique()[:3]
source = 'sweep'
print(distances, degrees)

fig, axs = plt.subplots(len(distances), len(degrees), sharex=True, sharey=True, squeeze=False)
fig.set_size_inches(10, 10*axs.shape[0]/axs.shape[1])

fig_psd, axs_psd = plt.subplots(len(distances), len(degrees), sharex=True, sharey=True, squeeze=False)
fig_psd.set_size_inches(10, 10*axs.shape[0]/axs.shape[1])
for i, distance in enumerate(distances):
    for j, degree in enumerate(degrees):
        df_this = df_total.loc[(df_total.distance == distance)
                               & (df_total.degree == degree)
                               & (df_total.source == source)]
        
        if not len(df_this) == 1:
            print('skipping', distance, degree, source)
            continue
        row = df_this.iloc[0]
        spec = row.spec #np.mean(row.spec, axis=1)
        psd = row.psd
        if psd is None:
            psd = get_psd(row.stft)
        
        axs[i, j].pcolorfast(range(spec.shape[0]), frequencies, np.log10(spec.T))
        for i_mic in range(psd.shape[0]):
            axs_psd[i, j].semilogy(frequencies, np.abs(psd[i_mic, :]), label=f"mic{i_mic}")
        axs[0, j].set_title(f'{degree} deg')
        axs_psd[0, j].set_title(f'{degree} deg')
    axs[i, 0].set_ylabel(f'{distance} cm')
    axs_psd[i, 0].set_ylabel(f'{distance} cm')
    
[axs[-1, j].set_xlabel(f'time idx') for j in range(len(degrees))]
[axs_psd[-1, j].set_xlabel(f'frequency [Hz]') for j in range(len(degrees))]

In [ ]:
from constants import SPEED_OF_SOUND

distances = df_total.distance.unique()[:3]
distance_ref = sorted(df_total.distance.unique())[-1]
source = 'sweep'
n_mics = df_total.iloc[0].stft.shape[1]
print('n_mics:', n_mics)
    
for degree in [0, 27, 54, 81][:2]:
    
    df_ref = df_total.loc[(df_total.distance == distance_ref)
                           & (df_total.degree == degree)
                           & (df_total.source == source)]
    if len(df_ref) == 0:
        continue

    fig, axs = plt.subplots(1, n_mics, sharex=True, sharey=True)
    fig.set_size_inches(15, 5)
    
    for mic in range(n_mics):
        psd_ref = df_ref.iloc[0].psd[mic] 
        for i, distance in enumerate(distances):
            df_this = df_total.loc[(df_total.distance == distance)
                                   & (df_total.degree == degree)
                                   & (df_total.source == source)]
            axs[mic].semilogy(frequencies, df_this.iloc[0].psd[mic], label=distance, color=f"C{i}")
            #axs[mic].semilogy(frequencies, df_this.iloc[0].psd[mic] / psd_ref, label=distance, color=f"C{i}")
            #axs[mic].plot(frequencies, df_this.iloc[0].psd[mic] - psd_ref, label=distance, color=f"C{i}")

        #axs[mic].set_xlim(min(frequencies), max(frequencies))
        axs[mic].set_xlim(2000,  max(frequencies))
        axs[mic].set_title(f"mic{mic}")
    
    fig.suptitle(degree)
    axs[mic].legend()

In [ ]:
#chosen_frequencies = frequencies[[10, 20, 30]]
chosen_frequencies = frequencies[[30]]
print(chosen_frequencies)

distances = df_total.distance.unique()

mic = 0

for degree, df in df_total.groupby('degree'):
    source = 'sweep'

    fig, ax = plt.subplots()
    fig.set_size_inches(15, 5)
    for i, distance in enumerate(distances):
        df_this = df.loc[(df.distance == distance)
                       & (df.source == source)]
        if len(df_this) == 0:
            continue
        row = df_this.iloc[0]
        for f, freq in enumerate(chosen_frequencies):
            chosen_idx = np.where(frequencies == freq)[0][0]
            ax.scatter(distance, row.psd[mic, chosen_idx], color=f"C{f}")
    ax.set_title(degree)
    #ax.set_yscale('log')
    ax.legend(chosen_frequencies)
    ax.set_xlabel('distance [cm]')

## Moving angle analysis

In [ ]:
#freq = 4125
#source = 'None'
source = 'mono4125'
degree = 360

if not degree in df_total.degree.unique():
    raise ValueError('cannot do moving angle analysis on this dataset')

distances = df_total.distance.unique()[:3]
chosen_idx = np.where(frequencies == freq)[0][0]

averages = []
for i, distance in enumerate(distances):
    fig, axs = plt.subplots(2)
    fig.set_size_inches(15, 5)
    
    ax = axs[0]
    df_this = df_total.loc[(df_total.distance == distance)
                           & (df_total.degree == degree)]
    df_this = df_this.loc[df_this.source == source]
                           
    stft = df_this.iloc[0].stft
    yaw =  df_this.iloc[0].yaw
    spec = df_this.iloc[0].spec

    for j in range(stft.shape[1]):
        ax.semilogy(range(stft.shape[0]), np.abs(stft[:, j, chosen_idx]), color=f"C{j}", label=f"mic{j}")

    yaw[np.isnan(yaw)] = 0
    axs[1].plot(range(stft.shape[0]), yaw)
    for deg in -np.arange(1, 5)*90:
        index = np.nanargmin(np.abs(yaw-deg))
        axs[1].axvline(x=index, color='C1')
        ax.axvline(x=index, color='C1')
        
    min_avg = 200
    max_avg = 300
    averages.append(np.sum(spec[min_avg:max_avg, chosen_idx], axis=0))
        
    ax.set_title(distance)
    ax.set_ylim(0.05, 10)
    ax.legend()
    [ax.grid() for ax in axs]
    
    
plt.figure()
plt.scatter(distances, averages)
plt.xlabel('distance [cm]')
plt.title(f'average PSD at {freq} Hz')